In [1]:
import pandas as pd
from wrongData import introduce_filthy_data

## Adding filthy data

In [2]:
df = pd.read_csv("./dataset/spotify_songs_restructured.csv", sep=",")
new_data = introduce_filthy_data(df, 1500)
new_data.to_csv("./dataset/spotify_songs_restructured_filthy.csv", index=False)
new_data.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,31167.000000,31225.000000,31283.000000,31224.000000,31183.000000,31220.000000,31154.000000,31163.000000,31180.000000,31153.00000,31259.000000,31198.000000,31135.000000
mean,42.528861,0.654592,0.698379,5.370548,-6.718501,0.564382,0.106937,0.175246,0.084761,0.19044,0.510586,120.816360,226007.054087
std,24.981296,0.145076,0.181060,3.612782,2.985200,0.495846,0.101163,0.219771,0.224078,0.15457,0.233095,26.900789,59903.953084
min,0.000000,0.000000,0.000175,0.000000,-46.448000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,4000.000000
25%,24.000000,0.563000,0.581000,2.000000,-8.166500,0.000000,0.041000,0.015100,0.000000,0.09280,0.331000,99.950000,187932.000000
50%,45.000000,0.671000,0.721000,6.000000,-6.170000,1.000000,0.062400,0.080400,0.000016,0.12700,0.513000,121.975000,216293.000000
75%,62.000000,0.760000,0.840000,9.000000,-4.650000,1.000000,0.131000,0.254000,0.004885,0.24900,0.693000,133.629000,253847.000000
max,100.000000,0.983000,1.000000,11.000000,1.275000,1.000000,0.918000,0.994000,0.994000,0.99600,0.991000,239.440000,517810.000000


In [3]:
new_data

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.0,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916,6.0,NaN,1.0,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754.0
1,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,NaN,0.726,0.815,11.0,-4.969,1.0,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600.0
2,All the Time - Don Diablo Remix,Zara Larsson,70.0,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675,0.931,1.0,NaN,0.0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616.0
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718,0.930,7.0,-3.778,1.0,NaN,0.028700,0.000009,0.2040,0.2770,121.956,169093.0
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.0,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,NaN,dance pop,0.650,0.833,1.0,-4.672,1.0,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,NaN,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,edm,progressive electro house,0.428,0.922,2.0,-1.814,1.0,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375.0
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20.0,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,edm,progressive electro house,NaN,0.786,0.0,-4.462,1.0,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120.0
32830,Sweet Surrender - Radio Edit,Starkillers,14.0,NaN,2014-04-21,♥ EDM LOVE 2020,edm,progressive electro house,0.529,0.821,6.0,-4.899,0.0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112.0
32831,Only For You - Maor Levi Remix,Mat Zo,15.0,Only For You (Remixes),2014-01-01,NaN,edm,progressive electro house,0.626,0.888,2.0,-3.361,1.0,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432.0


## Transforming the data

In [4]:
# nan van naam weghalen
df_clean = new_data.dropna(subset=['track_name', 'track_artist', 'track_album_name', 'track_album_release_date', 'playlist_name',
                                   'playlist_genre', 'playlist_subgenre'])
df_clean.head(10)

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.0,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916,6.0,NaN,1.0,0.0583,0.10200,0.000000,0.0653,0.518,122.036,194754.0
2,All the Time - Don Diablo Remix,Zara Larsson,70.0,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675,0.931,1.0,NaN,0.0,0.0742,0.07940,0.000023,0.1100,0.613,124.008,176616.0
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718,0.930,7.0,-3.778,1.0,NaN,0.02870,0.000009,0.2040,0.277,121.956,169093.0
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.0,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,Pop Remix,pop,dance pop,0.675,0.919,8.0,-5.385,1.0,0.1270,0.07990,0.000000,NaN,0.585,124.982,163049.0
6,Never Really Over - R3HAB Remix,Katy Perry,62.0,Never Really Over (R3HAB Remix),2019-07-26,Pop Remix,pop,dance pop,0.449,0.856,5.0,-4.788,0.0,0.0623,NaN,0.000000,0.1760,0.152,112.648,187675.0
7,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.0,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29,Pop Remix,pop,dance pop,0.542,0.903,4.0,-2.419,0.0,0.0434,0.03350,NaN,0.1110,0.367,127.936,207619.0
8,Tough Love - Tiësto Remix / Radio Edit,Avicii,68.0,Tough Love (Tiësto Remix),2019-06-14,Pop Remix,pop,dance pop,0.594,0.935,8.0,-3.562,1.0,0.0565,NaN,0.000004,0.6370,0.366,127.015,193187.0
10,Cross Me (feat. Chance the Rapper & PnB Rock) ...,Ed Sheeran,58.0,Cross Me (feat. Chance the Rapper & PnB Rock) ...,2019-06-21,Pop Remix,pop,dance pop,0.679,0.923,6.0,-6.500,1.0,0.1810,0.14600,NaN,0.1240,0.752,121.984,207894.0
13,SOS - Laidback Luke Tribute Remix / Radio Edit,Avicii,NaN,SOS (Laidback Luke Tribute Remix),2019-05-17,Pop Remix,pop,dance pop,0.572,0.915,5.0,-4.451,0.0,0.0625,0.01110,0.000000,0.3390,0.678,123.919,164516.0
14,Summer Days (feat. Macklemore & Patrick Stump ...,Martin Garrix,63.0,Summer Days (feat. Macklemore & Patrick Stump ...,2019-07-12,Pop Remix,pop,dance pop,0.690,NaN,5.0,-4.446,0.0,0.0594,0.00733,0.001830,0.0729,0.238,126.070,255238.0


### Imputing missing values

In [5]:
df_clean = df_clean.fillna({
    "danceability": df_clean["danceability"].mean(), 
    "energy": df_clean.energy.mean(), 
    "key": df_clean.key.mean(), 
    "loudness": df_clean.loudness.mean(), 
    "mode": df_clean['mode'].mean(), 
    "speechiness": df_clean.speechiness.mean(), 
    "acousticness": df_clean.acousticness.mean(), 
    "instrumentalness": df_clean.instrumentalness.mean(), 
    "liveness": df_clean.liveness.mean(), 
    "valence": df_clean.valence.mean(), 
    "tempo": df_clean.tempo.mean(), 
    "duration_ms": df_clean.duration_ms.mean(),
    "track_popularity": df_clean.track_popularity.mean()
})

df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,21284.000000,21284.000000,21284.000000,21284.000000,21284.000000,21284.000000,21284.000000,21284.000000,21284.000000,21284.000000,21284.000000,21284.000000,21284.000000
mean,42.511265,0.654843,0.698215,5.340483,-6.713226,0.569366,0.106659,0.176028,0.084660,0.191163,0.511229,120.775837,225844.188084
std,24.025910,0.139537,0.174283,3.467031,2.874346,0.476044,0.096705,0.212045,0.214847,0.149327,0.223997,25.822575,57979.172317
min,0.000000,0.000000,0.000175,0.000000,-36.624000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4000.000000
25%,27.000000,0.573000,0.595000,2.000000,-7.930000,0.000000,0.042500,0.018500,0.000000,0.095500,0.345000,100.161500,190043.250000
50%,42.511265,0.656000,0.702000,5.340483,-6.430500,1.000000,0.068700,0.100000,0.000040,0.139000,0.511229,120.775837,221173.000000
75%,61.000000,0.752000,0.831000,8.000000,-4.776750,1.000000,0.120000,0.231000,0.028900,0.232000,0.677000,130.065000,249479.250000
max,99.000000,0.983000,1.000000,11.000000,1.275000,1.000000,0.877000,0.994000,0.994000,0.996000,0.991000,220.252000,517125.000000


### outliers removal

In [6]:
# Define the columns where you want to remove outliers
columns_to_clean = ["danceability", "energy", "key", "loudness", "mode", 
                    "speechiness", "acousticness", "instrumentalness", 
                    "liveness", "valence", "tempo", "duration_ms"]

# Loop through each column and apply the custom quantile-based outlier removal
for col in columns_to_clean:
    Q01 = df_clean[col].quantile(0.001)  # 0.1st percentile
    Q99 = df_clean[col].quantile(0.999)  # 99.9th percentile
    
    # Remove rows where the values in the column are outside the 1st and 99th percentile bounds
    df_clean = df_clean[(df_clean[col] >= Q01) & (df_clean[col] <= Q99)]

# Display the cleaned DataFrame
df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000
mean,42.601086,0.656037,0.699587,5.343120,-6.661631,0.568528,0.106283,0.173557,0.081111,0.190537,0.512341,120.756719,225837.956944
std,24.062577,0.137662,0.171139,3.469121,2.734578,0.476191,0.094688,0.207642,0.209271,0.146977,0.222595,25.521834,56575.990753
min,0.000000,0.168000,0.074700,0.000000,-20.384000,0.000000,0.024100,0.000009,0.000000,0.021900,0.035300,63.248000,75333.000000
25%,27.000000,0.575000,0.596000,2.000000,-7.888250,0.000000,0.042600,0.018600,0.000000,0.095600,0.347000,100.267750,190290.250000
50%,43.000000,0.657000,0.703000,5.340483,-6.406000,1.000000,0.068900,0.099200,0.000036,0.139000,0.511229,120.775837,221173.000000
75%,61.000000,0.753000,0.831000,8.000000,-4.770000,1.000000,0.120000,0.229000,0.023925,0.231250,0.678000,130.051250,249188.000000
max,99.000000,0.964000,0.994000,11.000000,-0.958000,1.000000,0.604000,0.968000,0.958000,0.970000,0.972000,205.749000,503500.000000


### Categorical Encoding

In [7]:
# Corrected bins for track_popularity (7 edges, 6 labels)
popularity_bins = [0, 10, 30, 50, 70, 90, 100]
popularity_labels = ['Very Low', 'Low', 'Medium-Low', 'Medium', 'High', 'Very High']
df_clean['popularity_category'] = pd.Categorical(pd.cut(df_clean['track_popularity'], bins=popularity_bins, labels=popularity_labels))
# Add 'Unknown' to the categories
df_clean['popularity_category'] = df_clean['popularity_category'].cat.add_categories(['Unknown'])
# Update the rows where track_popularity is 0.0 to have the label 'Unknown'
df_clean.loc[df_clean['track_popularity'] == 0.0, 'popularity_category'] = 'Unknown'

tempo_bins = [60, 80, 100, 120, 140, 160, 180, 205]
tempo_labels = ['Very Slow', 'Slow', 'Moderate', 'Fast', 'Very Fast', 'Extremely Fast', 'Ultra Fast']
df_clean['tempo_category'] = pd.Categorical(pd.cut(df_clean['tempo'], bins=tempo_bins, labels=tempo_labels))

loudness_bins = [-21, -15, -10, -5, 0]
loudness_labels = ['Very Quiet', 'Quiet', 'Moderate', 'Loud']
df_clean['loudness_category'] = pd.Categorical(pd.cut(df_clean['loudness'], bins=loudness_bins, labels=loudness_labels))
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.000000,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748000,0.916000,...,0.058300,0.102000,0.000000,0.065300,0.5180,122.036,194754.0,Medium,Fast,Moderate
2,All the Time - Don Diablo Remix,Zara Larsson,70.000000,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675000,0.931000,...,0.074200,0.079400,0.000023,0.110000,0.6130,124.008,176616.0,Medium,Fast,Moderate
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.000000,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718000,0.930000,...,0.106659,0.028700,0.000009,0.204000,0.2770,121.956,169093.0,Medium,Fast,Loud
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.000000,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,Pop Remix,pop,dance pop,0.675000,0.919000,...,0.127000,0.079900,0.000000,0.191163,0.5850,124.982,163049.0,Medium,Fast,Moderate
6,Never Really Over - R3HAB Remix,Katy Perry,62.000000,Never Really Over (R3HAB Remix),2019-07-26,Pop Remix,pop,dance pop,0.449000,0.856000,...,0.062300,0.176028,0.000000,0.176000,0.1520,112.648,187675.0,Medium,Moderate,Loud
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32825,Booyah - Radio Edit,Showtek,42.511265,Booyah,2013-01-01,♥ EDM LOVE 2020,edm,progressive electro house,0.559000,0.916000,...,0.062600,0.045300,0.000013,0.225000,0.1950,128.012,215295.0,Medium-Low,Fast,Loud
32826,Wasted,Tiësto,47.000000,Wasted,2014-04-22,♥ EDM LOVE 2020,edm,progressive electro house,0.645000,0.698215,...,0.029400,0.001060,0.002640,0.199000,0.3750,112.028,188371.0,Medium-Low,Moderate,Moderate
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.511265,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,edm,progressive electro house,0.428000,0.922000,...,0.093600,0.076600,0.000000,0.066800,0.2100,128.170,204375.0,Medium-Low,Fast,Loud
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20.000000,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,edm,progressive electro house,0.654843,0.786000,...,0.042000,0.001710,0.004270,0.375000,0.4000,128.041,353120.0,Low,Fast,Loud


In [8]:
# release_date had NaT values die niet met dropna() verwijderd konden worden
df_clean['track_album_release_date'] = df_clean['track_album_release_date'].fillna(0)
df_clean = df_clean[df_clean['track_album_release_date'] != '0']

# sommige rijen hebben alleen jaar, mag niet
df_clean = df_clean[df_clean['track_album_release_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]

df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,19655.000000,19655.000000,19655.000000,19655.000000,19655.000000,19655.000000,19655.000000,19655.000000,19655.000000,19655.000000,19655.000000,19655.000000,19655.000000
mean,42.881415,0.658473,0.700184,5.337770,-6.579303,0.564623,0.107431,0.173864,0.082990,0.190213,0.507597,120.847550,223987.342107
std,24.020020,0.136482,0.170728,3.473178,2.685718,0.476836,0.095171,0.207659,0.212144,0.146332,0.222312,25.474137,55751.466704
min,0.000000,0.170000,0.074700,0.000000,-20.384000,0.000000,0.024100,0.000009,0.000000,0.021900,0.035300,63.248000,75333.000000
25%,27.000000,0.579000,0.598000,2.000000,-7.781500,0.000000,0.043200,0.018700,0.000000,0.095900,0.343000,100.611500,189077.500000
50%,43.000000,0.660000,0.703000,5.340483,-6.337000,1.000000,0.070300,0.100000,0.000036,0.138000,0.511229,120.775837,219320.000000
75%,61.000000,0.754000,0.831000,8.000000,-4.722000,1.000000,0.123000,0.229000,0.026000,0.231000,0.671500,130.037000,246174.000000
max,99.000000,0.964000,0.994000,11.000000,-0.958000,1.000000,0.604000,0.968000,0.958000,0.970000,0.972000,205.749000,503500.000000


## Feature splitting for release_date (Domain knowledge)

In [9]:
# Convert track_album_release_date to datetime, coercing errors
date_month = pd.to_datetime(df_clean['track_album_release_date'], errors='coerce')

# Use .loc to assign the new 'release_month' column safely
df_clean.loc[:, 'release_month'] = date_month.dt.strftime('%B')

df_clean.drop(columns=['playlist_subgenre', 'playlist_name'], inplace=True)
# df_clean.drop(columns=['instrumentalness', 'acousticness', 'mode', 'playlist_subgenre', 'playlist_name'], inplace=True)
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.000000,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,pop,0.748000,0.916000,6.0,-6.713226,...,0.102000,0.000000,0.065300,0.5180,122.036,194754.0,Medium,Fast,Moderate,June
2,All the Time - Don Diablo Remix,Zara Larsson,70.000000,All the Time (Don Diablo Remix),2019-07-05,pop,0.675000,0.931000,1.0,-6.713226,...,0.079400,0.000023,0.110000,0.6130,124.008,176616.0,Medium,Fast,Moderate,July
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.000000,Call You Mine - The Remixes,2019-07-19,pop,0.718000,0.930000,7.0,-3.778000,...,0.028700,0.000009,0.204000,0.2770,121.956,169093.0,Medium,Fast,Loud,July
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.000000,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,pop,0.675000,0.919000,8.0,-5.385000,...,0.079900,0.000000,0.191163,0.5850,124.982,163049.0,Medium,Fast,Moderate,July
6,Never Really Over - R3HAB Remix,Katy Perry,62.000000,Never Really Over (R3HAB Remix),2019-07-26,pop,0.449000,0.856000,5.0,-4.788000,...,0.176028,0.000000,0.176000,0.1520,112.648,187675.0,Medium,Moderate,Loud,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32825,Booyah - Radio Edit,Showtek,42.511265,Booyah,2013-01-01,edm,0.559000,0.916000,11.0,-3.050000,...,0.045300,0.000013,0.225000,0.1950,128.012,215295.0,Medium-Low,Fast,Loud,January
32826,Wasted,Tiësto,47.000000,Wasted,2014-04-22,edm,0.645000,0.698215,2.0,-5.595000,...,0.001060,0.002640,0.199000,0.3750,112.028,188371.0,Medium-Low,Moderate,Moderate,April
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.511265,City Of Lights (Vocal Mix),2014-04-28,edm,0.428000,0.922000,2.0,-1.814000,...,0.076600,0.000000,0.066800,0.2100,128.170,204375.0,Medium-Low,Fast,Loud,April
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20.000000,Closer Remixed,2013-03-08,edm,0.654843,0.786000,0.0,-4.462000,...,0.001710,0.004270,0.375000,0.4000,128.041,353120.0,Low,Fast,Loud,March


In [10]:
df_clean.to_csv("./dataset/spotify_songs_restructured_cleaned.csv", index=False)